#                                 Prédiction vecteur déplacement

Dans ce notebook nous allons utiliser la dataset TRUSS pour  proposer un modèle de réseaux de neurones artificiels, dans le but de prédire le vecteur déplacement à partir du vecteur aires des sections transversales (csa) .

## Importation des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

## Charger les données à partir du fichier Excel

In [2]:
chemin_fichier = "Truss.xlsx"
df = pd.read_excel(chemin_fichier)

## Préparation des données

In [3]:
X = df[['csa1','csa2','csa3','csa4','csa5','csa6','csa7','csa8','csa9','csa10',]]  # Variables d'entrée
y = df[['dx1', 'dy1', 'dx2', 'dy2', 'dx3', 'dy3', 'dx4', 'dy4', 'dx5', 'dy5', 'dx6', 'dy6']]  # Vecteur de déplacement

## Diviser le dataset en esemble de training et testsets 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Diviser le training_set en training and validation_sets


In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [6]:
print("Training set size: ", len(X_train))
print("Validation set size: ", len(X_val))
print("Test set size: ", len(X_test))

Training set size:  12800
Validation set size:  3200
Test set size:  4000


## Definir notre  modèle séquentiel

Le modèle séquentiel est composé par:
une première couche dense a 64 neurones avec une fonction d'activation ReLU, ensuite on ajoute une couche de dropout au modèle pour éviter le surapprentissage,puis on ajoute une autre couche dense au modèle avec 64 neurones et une fonction d'activation ReLU et le droupout, finalement une dernière couche dense au modèle avec 12 neurones. La fonction d'activation utilisée ici est softmax.

In [7]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

## Entrainement du modèle

In [8]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                704       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                780       
Total params: 5,644
Trainable params: 5,644
Non-trainable params: 0
_________________________________________________________________


In [9]:
history=model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_val, y_val))

Epoch 1/10
100/100 [==============================] - 0s 4ms/step - loss: 0.0144 - acc: 0.6302 - val_loss: 0.0136 - val_acc: 0.9834
Epoch 2/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0139 - acc: 0.5558 - val_loss: 0.0135 - val_acc: 0.9834
Epoch 3/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0138 - acc: 0.6313 - val_loss: 0.0135 - val_acc: 0.9834
Epoch 4/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0138 - acc: 0.6775 - val_loss: 0.0135 - val_acc: 0.9834
Epoch 5/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0138 - acc: 0.7396 - val_loss: 0.0134 - val_acc: 0.9834
Epoch 6/10
100/100 [==============================] - 0s 2ms/step - loss: 0.0137 - acc: 0.8006 - val_loss: 0.0134 - val_acc: 0.9834
Epoch 7/10
100/100 [==============================] - 0s 3ms/step - loss: 0.0137 - acc: 0.8573 - val_loss: 0.0134 - val_acc: 0.9834
Epoch 8/10
100/100 [==============================] - 0s 3ms/step - loss: 0.

Nous atteignons rapidement une précision de 0,958 (95,8 %) sur les données d'apprentissage. Vérifions maintenant que le modèle est également performant sur l'ensemble de test. 

## Evaluer  le modèle

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_acc)


Test accuracy: 0.9890000224113464


La précision de l'ensemble de test s'avère être de 98,9 %, ce qui est bien supérieur à la précision de l'ensemble de formation. d'entraînement. Cet écart entre la précision de l'apprentissage et celle du test est un exemple de l'overfitting.

##  Utiliser le réseau entraîné pour générer des prédictions sur de nouvelles données

In [19]:
X_new=np.array(X_test)
y_new=np.array(y_test)

In [20]:
predictions = model.predict(X_new)

In [21]:
print("Vecteur déplacement prédit :")
print(predictions)

Vecteur déplacement prédit :
[[0.11372223 0.11363818 0.14920935 ... 0.04327709 0.0685018  0.00443901]
 [0.09945638 0.09963758 0.10713687 ... 0.07362299 0.0838905  0.02873012]
 [0.1064285  0.10649885 0.12311196 ... 0.06201322 0.07982327 0.01417074]
 ...
 [0.10460751 0.10467707 0.11821874 ... 0.06568324 0.08146036 0.01730314]
 [0.09952505 0.09970202 0.10734111 ... 0.07352314 0.08388144 0.02849596]
 [0.09921848 0.09939323 0.1067853  ... 0.07375889 0.08383111 0.02947892]]


## calculer la métrique de l'erreur absolue moyenne (mean absolute error, MAE)

In [22]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
mean_absolute_error(y_new,predictions)

0.10789342333361694

En moyenne, les prédictions diffèrent de 0.10  unité par rapport aux valeurs réelles. Cela indique une bonne capacité de prédiction, car les différences entre les prédictions et les valeurs réelles sont relativement faibles.